In [ ]:
library(httr)
library(jsonlite)
library(foreach)
library(iterators)

In [ ]:
compare_dataframe <- function(comparison_id) {
    data <- fetch_data(comparison_id)
    return(create_dataframe(data))
}

fetch_data <- function(comparison_id) {
    path <- paste('https://www.orkg.org/orkg/simcomp/visualization/?resourceId=', comparison_id, sep='')
    r <- GET(url=path)
    r <- content(r, as='text', encoding='UTF-8')
    df <- fromJSON(r,flatten = TRUE)
    path <- paste('https://www.orkg.org/orkg/simcomp/compare/', df$data$url, sep='')
    r <- GET(url=path)
    r <- content(r, as='text', encoding='UTF-8')
    return(fromJSON(r,flatten = TRUE))
}

create_dataframe <- function(data) {
    contributions <- data$contributions
    properties <- data$properties

    property_labels <- properties['label']
    contribution_labels <- foreach(d=iter(contributions, by='row'), .combine='rbind') %do% paste(d[['title']], d[['contributionLabel']], sep='/')

    df <- data.frame(matrix(nrow = nrow(property_labels), ncol = nrow(contribution_labels)))
    rownames(df) <- property_labels$label
    colnames(df) <- contribution_labels

    d1 <- data$data

    for(i in 1:length(d1)) {
        d2 <- d1[[i]]
        for(j in 1:length(d2)) {
            d3 <- d2[[j]]
            label <- d3[['label']]
            contribution_id <- d3[['path']][[1]][[1]]

            if (is.null(contribution_id))
                next

            contribution <- contributions[contributions['id'] == contribution_id, ]
            contribution_label <- paste(contribution$title, contribution$contributionLabel, sep='/')
            property_id <- d3[['path']][[1]][[length(d3[['path']][[1]])]]
            property_label <- properties[properties['id'] == property_id, ]$label
        
            if(length(label) == 1)
                df[property_label, contribution_label] <- label
            else
                df[property_label, contribution_label] <- paste(label, collapse = "/")
        }
    }
    
    return(df)
}

In [ ]:
df <- compare_dataframe(comparison_id='R53407')

t(df)